In [49]:
import json
from litellm import completion

LLM_MODEL = "gpt-4o"

def call_llm_get_json(prompt, query):
    return json.loads(completion(
            model=LLM_MODEL,
            messages=[
                {
                    "content": prompt,
                    "role": "system",
                },
                {
                    "content": query,
                    "role": "user",
                },
            ],
            response_format = { "type": "json_object" }
        ).choices[0].message.content)

def call_llm(prompt, query):
    return completion(
            model=LLM_MODEL,
            messages=[
                {
                    "content": prompt,
                    "role": "system",
                },
                {
                    "content": query,
                    "role": "user",
                },
            ],
        ).choices[0].message.content

In [37]:
def get_translation(original_query, message):
        prompt = f"""Please translate the following user message from English into the language of this original query: f{original_query}. The message to translate follows."""
        return call_llm(prompt, message)

In [63]:
def get_user_intent(query):
    prompt = """
You're supporting users of the Benefit Navigator tool, which is an online tool, "one-stop shop," case managers use when working with individuals and families to help them understand, access, and navigate the complex public benefits and tax credit landscape in the Los Angeles region.

Please analyze the user message and respond with a JSON object that matches the following schema:
```
{
    /**
     * If the user is trying to understand what benefit programs the chatbot supports, for example, “What do you know about?", "What info do you have?", "What can I ask you?" "What programs do you cover?", "What benefits do you cover?", "What topics do you know?", set user_intent to “benefits_covered”.
     * If the user's question is about how to reset their password for the Benefit Navigator, set user_intent to "reset_password”.
     * If the user’s question is about these topics related to the benefit navigator: changing phone number for two-factor authentication, trouble creating or saving clients, trouble creating or saving reports, trouble finding clients in the user portal, or any other kinds of support questions for the benefit navigator tool, then set user_intent to “navigator_tool”.
     * For any other kind of question – for example, about accessing public benefits or getting assistance with programs, set user_intent to null.
     */
    user_intent: "benefits_covered" | "reset_password" | "navigator_tool" | null;

  /**
   * The name of the in-scope benefit program that the user's question is about,
   * or null if the benefit program is not listed one of the in-scope programs listed below
   */
  benefit_program: string | null;

  /**
   * The English translation of the user's question if it was not in English,
   * or null if the user's question was already in English
   */
  message_in_english: string | null;

  /**
   * Indicates whether searching through policy documentation would help
   * to answer the user's question
   * 
   * true: Additional context from documentation is needed
   * false: No additional documentation is required (e.g., for translation requests)
   */
  needs_context: boolean;

    /**
      * Set `topic` to one of the following string values that matches the PRIMARY topic of the user's questions, or null if none match. For a topic to match and be primary, the question must be directly about assistance with the topic itself, not about something else. For example, a question about whether a passport was an acceptable ID cards for food stamps would NOT have the topic "passport", since this question is actually about food stamps.
      */
    topic =
  | "ID cards"
  | "passport"
  | "birth certificates"
  | "social security number"
  | "ITIN"
  | "applying for citizenship"
  | "applying for green card"
  | "transit cards"
  | "DPSS contact info or locations"
  | "DPSS appeals"
  | "transportation for people with disabilities"
  | "food banks"
  | "wildfire resources"
  | "benefit navigator screening"
  | "LA county hospitals and clinics"
  | "LGBTQ resources"
  | "med-cal for immigrants"
  | "Medi-Cal asset limits"
  | "CalFresh work requirements (ABAWDs, time limits)"
  | "Calfresh asset limits/resource limits"
  | null;
}

In-scope benefit programs include:
- CalWORKS (including CalWORKS childcare)
- General Relief,
- Housing programs: CalWORKS Homeless Assistance (HA) for Permanent HA, Permanent HA Arrerages, Expanded Temporary HA, \
CalWORKS WtW Housing Assistance, including Emergency Assistance to Prevent Eviction (EAPE), \
Temporary Homeless Assistance Program (THAP or Temporary HA) + 14, CalWORKS Homeless Assistance (HA): Permanent HA,  Moving Assistance (MA), \
4 Month Rental Assistance, General Relief (GR) Rental Assistance, General Relief (GR) Move-In Assistance, \
Crisis/Bridge Housing, Access Centers, Outreach Services, Family Solutions Center,
- CalFresh, WIC,
- Medi-Cal (Medicaid), ACA (Covered California)
- CARE, FERA, LADWP EZ-Save, LifeLine,
- Tax credits: Earned Income Tax Credit (EITC), California Earned Income Tax Credit (CalEITC), \
Child Tax Credit (CTC) and Additional Child Tax Credit, Young Child Tax Credit,  California Child and Dependent Care Tax Credit, \
Child and Dependent Care Tax Credit (CDCTC), California Renter's Credit, California Foster Youth Tax Credit,
- Supplemental Security Income (SSI), Social Security Disability Insurance (SSDI),
- SDI (State Disability Insurance),
- Veterans Benefits (VA),
- Cash Assistance Program for Immigrants (CAPI)
- Public Charge,
- In-Home Supportive Services,
- EDD programs: Unemployment insurance (UI), state disability insurance (SDI), paid family leave (PFL)
```
"""
    return call_llm_get_json(prompt, query)

In [64]:
import timeit
import concurrent.futures

from src.chat_engine import ImagineLaEngine, ImagineLA_MessageAttributes
from src.generate import analyze_message


def monolithic(TEST_QUERY):
    analyze_message(
            LLM_MODEL, ImagineLaEngine.system_prompt_1, TEST_QUERY, response_format=ImagineLA_MessageAttributes
        )

def modular(TEST_QUERY):
    attr = get_user_intent(TEST_QUERY)

    if attr["user_intent"]:
        if attr["message_in_english"]:
            return get_translation(TEST_QUERY, "This is a pre-programmed response.")
        else:
            return "This is a pre-programmed response."

    if attr["topic"] in ["ID cards", "passport"]: # all of the referral categories
        if attr["message_in_english"]:
            return get_translation(TEST_QUERY, "This is a pre-programmed referral link.")
        else:
            return "This is a pre-programmed referral link."

qs = [
    "I need help with my password",
    "I can't find client accounts.",
    "Where do I apply for a passport?",
    "How do I get food stamps?" # <- This last one is often slower for the JSON, why?
] 

N = 1
for q in qs:
    print(q)
    print("Text response: ", timeit.timeit(stmt=lambda: monolithic(q), number=N))
    print("JSON response: ", timeit.timeit(stmt=lambda: modular(q), number=N))
    print()

I need help with my password
Monolithic:  1.9953071258496493
Modular:  0.702693291939795

I can't find client accounts.
Monolithic:  1.2076422090176493
Modular:  0.8623478750232607

Where do I apply for a passport?
Monolithic:  1.8157232920639217
Modular:  0.9181146260816604

How do I get food stamps?
Monolithic:  0.7480586250312626
Modular:  0.8099462089594454

